# Multi-Asset Dashboard: Equity and Fixed Income Integration

This notebook integrates equity and fixed income strategy metrics to enable cross-asset decision making. We apply a consistent analytics framework—cumulative and annualized returns, volatility, Sharpe ratio, and drawdowns—across both asset classes, then merge results for joint ranking, visualization, and allocation guidance.

In [ ]:
# Imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

DATA_EQUITY = Path('../data/strategy_returns.csv')
DATA_FIXED = Path('../data/fixed_income_strategy_returns.csv')

assert DATA_EQUITY.exists(), f'Equity data not found: {DATA_EQUITY}'
assert DATA_FIXED.exists(), f'Fixed income data not found: {DATA_FIXED}'


In [ ]:
# Helper functions for performance metrics
def to_periods_per_year(index):
    if isinstance(index, pd.DatetimeIndex):
        if index.inferred_freq is not None:
            freq = index.inferred_freq.upper()
            if freq.startswith('D'): return 252
            if freq.startswith('W'): return 52
            if freq.startswith('M'): return 12
            if freq.startswith('Q'): return 4
            if freq.startswith('A') or freq.startswith('Y'): return 1
        # Fallback by density
        counts = len(index) / ((index.max() - index.min()).days / 365.25)
        return int(np.clip(round(counts), 1, 252))
    return 252

def cum_return(r):
    return (1 + r).prod() - 1

def ann_return(r, periods_per_year=None):
    n = r.shape[0]
    if periods_per_year is None:
        periods_per_year = to_periods_per_year(r.index) if hasattr(r, 'index') else 252
    total = (1 + r).prod()
    years = n / periods_per_year if periods_per_year else 1
    return total**(1/years) - 1 if years > 0 else np.nan

def ann_vol(r, periods_per_year=None):
    if periods_per_year is None:
        periods_per_year = to_periods_per_year(r.index) if hasattr(r, 'index') else 252
    return r.std(ddof=0) * np.sqrt(periods_per_year)

def sharpe(r, rf=0.0, periods_per_year=None):
    if periods_per_year is None:
        periods_per_year = to_periods_per_year(r.index) if hasattr(r, 'index') else 252
    excess = r - rf / periods_per_year
    vol = ann_vol(r, periods_per_year)
    return np.where(vol != 0, (ann_return(r, periods_per_year) - rf) / vol, np.nan)

def max_drawdown(r):
    wealth = (1 + r).cumprod()
    peak = wealth.cummax()
    dd = (wealth / peak) - 1
    return dd.min()

def summarize(df_returns, label_prefix=None):
    ppy = to_periods_per_year(df_returns.index)
    metrics = pd.DataFrame({
        'Cumulative Return': df_returns.apply(cum_return),
        'Annualized Return': df_returns.apply(ann_return, periods_per_year=ppy),
        'Annualized Volatility': df_returns.apply(ann_vol, periods_per_year=ppy),
        'Sharpe Ratio': df_returns.apply(sharpe, periods_per_year=ppy),
        'Max Drawdown': df_returns.apply(max_drawdown)
    })
    if label_prefix:
        metrics.index = [f'{label_prefix} | {c}' for c in metrics.index]
    return metrics


In [ ]:
# Load data
def load_returns(path):
    df = pd.read_csv(path)
    # Expect a date column and one column per strategy with returns in decimal (e.g., 0.01)
    date_col = None
    for c in df.columns:
        if 'date' in c.lower():
            date_col = c
            break
    if date_col is None:
        raise ValueError('No date column found in returns file: ' + str(path))
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    # Convert percentage returns if values look like percents
    vals = df.select_dtypes(include=[np.number])
    if ((vals.abs() > 1).mean() > 0.5).any():
        df[vals.columns] = vals / 100.0
    return df

eq = load_returns(DATA_EQUITY)
fi = load_returns(DATA_FIXED)
# Align by inner join to the common date range
eq, fi = eq.align(fi, join='inner', axis=0)
print(eq.shape, fi.shape)


In [ ]:
# Compute summaries using shared metric functions
summary_eq = summarize(eq, label_prefix='Equity')
summary_fi = summarize(fi, label_prefix='Fixed Income')
summary_all = pd.concat([summary_eq, summary_fi], axis=0)
# Rank by Sharpe as default
summary_all_sorted = summary_all.sort_values('Sharpe Ratio', ascending=False)
summary_all_sorted.head()


In [ ]:
# Joint bar charts for key metrics
metrics_to_plot = ['Annualized Return', 'Annualized Volatility', 'Sharpe Ratio', 'Max Drawdown']
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
axes = axes.flatten()
for ax, m in zip(axes, metrics_to_plot):
    plot_df = summary_all.copy()
    if m == 'Max Drawdown':
        plot_df = plot_df.sort_values(m)  # more negative worse
    else:
        plot_df = plot_df.sort_values(m, ascending=False)
    plot_df[m].plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title(m)
    ax.axhline(0, color='black', linewidth=0.8)
    ax.set_xlabel('Strategy')
    ax.set_ylabel(m)
    ax.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# Risk-return scatter for all strategies
ppy = to_periods_per_year(eq.index)
all_returns = pd.concat([eq.add_prefix('Equity | '), fi.add_prefix('Fixed Income | ')], axis=1)
ann_ret = all_returns.apply(ann_return, periods_per_year=ppy)
ann_vol_ = all_returns.apply(ann_vol, periods_per_year=ppy)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=ann_vol_, y=ann_ret)
for s in all_returns.columns:
    plt.annotate(s, (ann_vol_[s], ann_ret[s]), textcoords='offset points', xytext=(5,5))
plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Return')
plt.title('Risk-Return: Equity vs Fixed Income Strategies')
plt.grid(True, linestyle='--', alpha=0.4)
plt.show()


In [ ]:
# Correlation matrix across all strategies
corr = all_returns.corr()
plt.figure(figsize=(12, 9))
sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
plt.title('Correlation Matrix: Equity and Fixed Income Strategies')
plt.tight_layout()
plt.show()
corr.head()


## Manager Guidance: Cross-Asset Allocation and Tradeoffs

- Use the Sharpe-ranked summary to overweight strategies with superior risk-adjusted returns across both asset classes.
- Combine low-correlated strategies (see correlation heatmap) to enhance diversification and reduce portfolio volatility.
- Balance drawdown tolerance: pair high-return equity strategies with stable fixed income sleeves to cap portfolio max drawdown.
- In risk-on regimes, tilt toward higher-return, acceptable-volatility equity strategies; in risk-off regimes, rotate toward resilient fixed income strategies.
- Target allocations can be optimized via mean-variance on the full strategy set or by simple risk-budgeting (e.g., equal risk contribution).
- Monitor rolling Sharpe and drawdowns to adapt allocations as regime changes emerge.
